# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847673633630                   -0.70    4.5   28.8ms
  2   -7.852385195870       -2.33       -1.53    1.0   17.0ms
  3   -7.852608698699       -3.65       -2.54    1.2   17.4ms
  4   -7.852645444228       -4.43       -2.77    2.5   22.1ms
  5   -7.852646087138       -6.19       -2.87    1.2   17.5ms
  6   -7.852646668034       -6.24       -3.85    1.0   17.1ms
  7   -7.852646684922       -7.77       -4.56    1.5   18.5ms
  8   -7.852646686679       -8.76       -5.05    1.8   19.8ms
  9   -7.852646686726      -10.34       -5.90    1.0   17.1ms
 10   -7.852646686727      -11.77       -5.61    2.2   22.0ms
 11   -7.852646686730      -11.61       -6.61    1.0   17.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847792325897                   -0.70           4.8    368ms
  2   -7.852561709296       -2.32       -1.62   0.80    2.0    2.20s
  3   -7.852641221041       -4.10       -2.71   0.80    1.0    222ms
  4   -7.852646510899       -5.28       -3.38   0.80    1.8   18.6ms
  5   -7.852646681665       -6.77       -4.40   0.80    1.5   17.6ms
  6   -7.852646686627       -8.30       -4.88   0.80    2.5   21.6ms
  7   -7.852646686726      -10.00       -5.54   0.80    1.2   16.8ms
  8   -7.852646686730      -11.46       -6.80   0.80    1.5   44.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.580126083218                   -1.03    4.38s
  2   -1.744853950801        0.52       -0.65    108ms
  3   -3.682796050944        0.29       -0.44   32.7ms
  4   -5.248133110475        0.19       -0.60   32.7ms
  5   -6.714268677196        0.17       -0.79   32.7ms
  6   -7.079485842840       -0.44       -1.54   24.6ms
  7   -7.566240589324       -0.31       -1.68   24.5ms
  8   -7.698053644269       -0.88       -1.81   24.6ms
  9   -7.746166754830       -1.32       -1.86   24.6ms
 10   -7.775219637867       -1.54       -2.05   24.6ms
 11   -7.799130754917       -1.62       -2.03   24.6ms
 12   -7.828836285966       -1.53       -2.38   24.5ms
 13   -7.845434713962       -1.78       -2.33   24.6ms
 14   -7.849478717253       -2.39       -3.09   24.5ms
 15   -7.851775687990       -2.64       -2.99   24.5ms
 16   -7.852337763460       -3.25       -3.26   24.6ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847774730799                   -0.70    4.8   30.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645844827                   -1.63    13.5s
  2   -7.852646686730       -6.07       -3.69    3.55s
  3   -7.852646686730      -13.25       -7.20    124ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.604260338846249e-7
|ρ_newton - ρ_scfv| = 5.548967209614181e-7
|ρ_newton - ρ_dm|   = 7.923082878754927e-10
